In [46]:
import numpy as np
import qutip as qu
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import csv
import sympy as sp

In [47]:
#define constants
e = 1.602*10**(-19)#C
hbar = 1.054*10**(-34)#Js
h = hbar*2*np.pi
phi0 = hbar/2/e

Minimal notebook to obtain the effective qubit-qubit coupling $J$, Eq. (8), with the circuit in Fig. 7(a) and $C_k\to 0$. 

The objective is for you to get an idea of the overall workflow. All references to Eqs. are to our paper "Toolbox for Nonreciprocal dispersive models in cQED". 

If so desired, how to get the dispersive shifts, shifted resonator frequency, higher-order nonlinear corrections, and even the exact Hamiltonian can be added without any conceptual difficulties. In the same way the admittance can be added. 

# Circuit parameters

First, choose the circuit parameters, and any random $s$ to get the response somewhere else, in my case with Mathematica. 

In [39]:
# paths
cwd = os.getcwd()
path_params = cwd + '/Params-files/'
print(path_params)

# define the circuit parameters 
Ej = 20.0*2*np.pi # Josephson energy (GHz)
Lj = phi0**2/Ej/hbar # Josephson inductance (nH)
wq = 5.0*2*np.pi # qubit frequency (GHz)
Cj = 1/(wq**2*Lj) # qubit capacitance (nF)
Ec = e**2/2/Cj/hbar # Charging energy (GHz)
Cc = 0.05 * Cj # Coupling capacitance
wr = 2*np.pi*6.0 # resonator frequency (GHz)
zr = 50.0 # resonator impedance (Ohm)
Cr = 1/(wr*zr) # resonator capacitance (nF)
Lr = zr/wr # resonator inductance (nH)
s = 1.0 # just a dummy variable for the moment
print('Lj',Lj,'Cj',Cj*1e6, 1/np.sqrt(Lj*Cj)/2/np.pi, 'Ej/Ec', Ej/Ec)

list_params = [Cj,Cc,Cr,Lr,s]
name = 'params-file-'+str(1)+'.csv'
with open(path_params + name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(list_params)

/Users/lautarolabarca/Dropbox/SW_NR_BB_quantization/Codes/Simple-examples/Params-files/
Lj 8.170445398447482 Cj 124.00937611258065 5.0 Ej/Ec 127.99999999999999


# Load the capacitive matrix

The capacitive matrix is obtained from the 0-pole matrix. For the impedance (see Eq. A3), 
$$ A_0 = \lim_{s\to 0}sZ[s]$$

The code that follows is agnostic to how the matrix is obtained, simply store it and then load it here. 

We use this matrix to obtain the capacitance matrix. The diagonal entries correspond to the dressed capacitance the junction sees. 

The dressed capacitance is used to define the dressed and shifted to account for the lamb shift qubit frequency, Eq. 6

In [40]:
# Function to read a CSV file and convert it to a sympy matrix. Used to handle the imaginary part I, mathematica returns it symbolically. 
def read_csv_to_sympy_matrix(file_path):
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        matrix = []
        for row in reader:
            matrix.append([sp.sympify(cell.replace('e', '*10**')) for cell in row])
        return sp.Matrix(matrix)

# Function to read a CSV file and convert it to a numpy array
def read_csv_to_numpy_array(file_path):
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        matrix = []
        for row in reader:
            matrix.append([float(cell) for cell in row])
        return np.array(matrix)

index = 1
Path_RM = cwd + "/Response-matrices/"
Path_A0 = Path_RM + 'A0-'+str(index)+'.csv'
A0s = read_csv_to_numpy_array(Path_A0)
Cmat = np.linalg.inv(A0s)
CJ = Cmat[0,0] # dressed junction capacitance
Ec = e**2/2/CJ/hbar # Charging energy (GHz)

wj0 = 1/np.sqrt(Lj*CJ)
wj = wj0*(1-Ec/wj0/(1-Ec/wj0))

# Re-run the Mathematica notebook manually, to get the impedance matrix at the shifted qubit frequency

Get the impedance response matrix used in Eqs (5 and 8).

In [34]:
# import subprocess

# # wolframscript_path = "/Applications/Mathematica.app/Contents/MacOS/wolframscript"
# # I could add here code to iterate over many different parameters using wolframscript 
# # Sadly running wolframscript is quite slow, I do not understand the reason for this. 
# # For the paper I imported the analytical form of the impedance onto python with sympy, but I guess
# # that for real circuits that is no go. Hence, the workflow here mimics the situation 
# # where you simply run your impedance finder somewhere else, 
# # and then simply load it here to apply the formulas, find the exact Hamiltonian, etc. 

# mathematica_script = cwd + '/Get-Responses.wls'
# wolframscript_path = "/Applications/Mathematica.app/Contents/MacOS/wolframscript"
# subprocess.run([wolframscript_path, "-file", mathematica_script])
# subprocess.run([wolframscript_path, "-file", mathematica_script])

In [43]:
s = wj # the imaginary part is added in the notebook. The frequency is given by Eq. (6). 
list_params = [Cj,Cc,Cr,Lr,s]
# Run the Mathematica Notebook, this I am doin manually, but it can be done automatically if wolframscript is installed
list_params = [Cj,Cc,Cr,Lr,s]
name = 'params-file-'+str(1)+'.csv'
with open(path_params + name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(list_params)

In [44]:
# Rerun the Mathematica and load the Z matrix
Path_Z = Path_RM + 'Z-'+str(index)+'.csv'
Z = read_csv_to_sympy_matrix(Path_Z) #the sympy is needed due to how Mathematica treats the imaginary number i
Z = np.array(Z).astype(np.complex64)
print('Z matrix',Z)

Z matrix [[0.-2.5838751e+02j 0.+2.4415778e-01j]
 [0.+2.4415778e-01j 0.-2.5838751e+02j]]


# Getting the effective qubit-qubit coupling

Direct application of Eq. (8)

In [45]:
J = 1j/4*np.sqrt(wj*wj/Lj/Lj)*(Z[0,1]/wj+Z[0,1]/wj)

print('resonator bare frequency wr (GHz)/2pi',wr/2/np.pi)
print('dressed junction capacitance CJ',CJ*1e6)
print('EJ/Ec', Ej/Ec)
print('bare qubit frequency wj0 (GHz)/2pi',wj0/2/np.pi)
print('shifted qubit frequency Eq. (6) wj (GHz)/2pi',wj/2/np.pi)
print('effective coupling J Eq. (8) (MHz)/2pi',J*1e3/2/np.pi)

resonator bare frequency wr (GHz)/2pi 6.0
dressed junction capacitance CJ 130.20984491820968
EJ/Ec 134.39999999999998
bare qubit frequency wj0 (GHz)/2pi 4.879500364742666
shifted qubit frequency Eq. (6) wj (GHz)/2pi 4.726009859841583
effective coupling J Eq. (8) (MHz)/2pi (-2.3780170533758866+0j)
